# Cleanup from csv
Some cleanup was performed in Excel, such as filling in some missing data, so this will not a completely replicate the data used in the dashboard.

In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt

In [ ]:
#read in the csv from data acquisition
plants = pd.read_csv('plants3.csv')
plants.info()

## Height and width

In [ ]:
# Split height and width columns
plants[['H1','W1']] = plants['Mature Size'].str.split('[,;]',expand=True)
plants['H1'] = plants['H1'].str.replace('Height: ', '')
plants['W1'] = plants['W1'].str.replace('Spread: ', '')
plants['Height'] = plants['Height'].fillna(plants['H1'])
plants['Width'] = plants['Height'].fillna(plants['W1'])

In [ ]:
plants = plants.drop(columns = ['Mature Size', 'H1', 'W1'])
plants.info()

## Blooming times

In [ ]:
# blooming times may not be needed, but are cleaned up a bit anyways
plants['Bloom season'] = plants['Bloom season'].fillna(plants['Bloom Time'])
plants = plants.drop(columns = 'Bloom Time')
plants.info()

In [ ]:
plants = plants.drop(columns = 'Unnamed: 0')

In [ ]:
# an error was found in the process...
plants.loc[plants['Botanical Name'].str.find('Delosperma') > -1]

In [ ]:
# This row is wrong because the website has an error
plants.loc[200, 'Botanical Name'] = 'Delosperma congestum'
plants.loc[200, 'Common Name'] = 'White Nugget'
plants.loc[200, 'Width'] = '8-12 inches'

In [ ]:
# removes common name from Botanical Name column
plants['Botanical Name'] = plants['Botanical Name'].fillna(plants['Common Name'])
plants['Botanical Name'] = plants['Botanical Name'].str.replace(r'‘.*’','', regex=True)

In [ ]:
# check for more 
plants.loc[plants['Common Name'].str.find('‘') > -1]

## Plant Types

In [ ]:
# Consolidate the plant types into reasonable categories
plants['Type'] = plants['Plant Type']
plants.loc[(plants['Family'].str.find('Cactaceae') > -1),'Type'] = 'Cactus'
plants['Type'] = plants['Type'].str.title()
plants['Type'] = plants['Type'].str.replace(',*\s*[Pp]erennial[;,]*\s*','',regex=True)
plants.loc[plants['Type'].isin(['Flowering Succulent','Succulent Evergreen','Succulent, House Plant']),'Type'] = 'Succulent'
plants['Type'] = plants['Type'].str.replace('Epiphytic Fern','Epiphyte')
plants.loc[(plants['Type'].str.len() < 3), 'Type'] = 'Other'

In [ ]:
plants.loc[plants['Type'] == 'Cactus – Succulents, Shrubs']

## Light Conditions

In [ ]:
# Condense the light conditions
plants['Light'] = plants['Light'].str.title()
plants.loc[plants['Light'].isin(['Full Sun, Partial Shade','Full Sun To Partial Shade','Full Sun, Partial Sun',
                                 'Full Sun, Part Shade','Full Sun Or Partial Shade','Part Shade To Full Sun',
                                 'Full Sun To Part Shade']),'Light'] = 'Full Sun To Part Shade'
plants.loc[plants['Light'].isin(['Partial Shade','Part Shade, Part Sun']),'Light'] = 'Part Shade'
plants['Light'] = plants['Light'].str.replace('Partial Or Dappled Shade','Light Shade to Part Shade')
plants['Light'] = plants['Light'].str.replace('Bright, Direct.*','Direct Light')
plants.loc[plants['Light'].isin(['Full Sun, Light Shade','Full Sun To Bright Shade','Full Sun, Bright Shade',
                                 'Full Sun Or Light Shade']),'Light'] = 'Full Sun to Light Shade'
plants.loc[plants['Light'].isin(['Part Shade To Full Shade','Part Shade Or Deep Shade']),'Light'] = 'Part Shade to Full Shade'
plants['Light'] = plants['Light'].str.replace('Sunlight','Light')
plants.loc[plants['Light'].isin(['Bright Light','Direct Light','Bright, Direct Light','Bright, Full Sun']),'Light'] = 'Full Sun'
plants['Light'] = plants['Light'].str.replace('Full Sun, Dappled Sun, Part Shade, Deep Shade','Full Sun to Full Shade')

# more light changes
plants.loc[(plants['Light'] == 'Full Sun, Dappled Sun, Partial Shade, Deep Shade'),'Light'] = 'Full Sun to Full Shade'
plants.loc[(plants['Light'] == 'Partial Shade To Full Shade'),'Light'] = 'Part Shade to Full Shade'
plants.loc[(plants['Light'] == 'Full Sun To Part Shade'),'Light'] = 'Full Sun to Part Shade'

In [ ]:
# Ok now I want to add a numerical column based on 100-Full Sun, 75- light shade, 50-Part Shade, 25-Full Shade 
plants['Minimum Light'] = 100
plants.loc[(plants['Light'].str.find('Light Shade') >-1),'Minimum Light'] = 75
plants.loc[(plants['Light'].str.find('Indirect') >-1),'Minimum Light'] = 75
plants.loc[(plants['Light'].str.find('Part Shade') >-1),'Minimum Light'] = 50
plants.loc[(plants['Light'].str.find('Full Shade') >-1),'Minimum Light'] = 25

In [ ]:
# and a maximum light column as well
plants['Maximum Light'] = 75
plants.loc[(plants['Light'].str.find('Full Sun') >-1),'Maximum Light'] = 100
plants.loc[(plants['Light'] == 'Part Shade'),'Maximum Light'] = 50
plants.loc[(plants['Light'] == 'Part Shade to Full Shade'),'Maximum Light'] = 50
plants.loc[(plants['Light'] == 'Bright, Indirect Light'),'Maximum Light'] = 100

## Water needs

In [ ]:
# Water needs is next
plants['Water needs'] = plants['Water needs'].str.replace('Low/Average','Low to Moderate')
plants['Water needs'] = plants['Water needs'].str.replace('Low, Moderate','Low to Moderate')
plants['Water needs'] = plants['Water needs'].str.replace('Low to Average','Low to Moderate')
plants['Water needs'] = plants['Water needs'].str.replace('Low/Moderate','Low to Moderate')

## Soil

In [ ]:
# Condense the soil types
plants['Soil'] = plants['Soil'].str.title()
plants['Soil'] = plants['Soil'].str.replace('\s?Well[-\s]Drained\s?','', regex=True)
plants['Soil'] = plants['Soil'].str.replace('\s?Good[-\s]Drainage\s?','', regex=True)
plants.loc[plants['Soil'].isin([', Loamy, Sandy',', Loam, Sand',', Sandy/Loamy','; Loam, Sand']),'Soil'] = 'Loamy, Sandy'
plants.loc[plants['Soil'].isin([', Loamy','Loamy, ','Loamy,']),'Soil'] = 'Loamy'
plants.loc[plants['Soil'].isin(['Moist, Well–Drained','Moist, ','Moist,',', Moist','Moist ',', Slightly Moist','Medium Moisture,']),'Soil'] = 'Moist'
plants['Soil'] = plants['Soil'].str.replace(',$','', regex=True)
plants['Soil'] = plants['Soil'].str.replace('Moist,, Fertile & Loamy','Moist, Fertile, Loamy')
plants['Soil'] = plants['Soil'].str.replace('Or ','or ')
plants.loc[plants['Soil'].isnull(), 'Soil'] = 'None'
plants.loc[(plants['Soil'] == ''),'Soil'] = 'Regular'
plants.loc[(plants['Soil'].isin(['Sandy','Occasionally Dry','Porous'])),'Soil'] = 'Loamy, Sandy'

## Tolerance

In [ ]:
# Condense drought tolerant and remove deer/rabbit as it's not relevant for target demographic
plants['Tolerant'] = plants['Tolerant'].str.replace(',?\s?Deer,?','', regex=True)
plants['Tolerant'] = plants['Tolerant'].str.replace(',?\s?Rabbit,?','', regex=True)
plants['Tolerant'] = plants['Tolerant'].str.replace('^\s','', regex=True)
plants.loc[plants['Tolerant'].isin(['Drought, Dry soil','Drought Dry Soil','Drought Rocky Soil',
                                    'Drought, Dry or Rocky Soil',' Drought, Rocky Soil']),'Tolerant'] = 'Drought'
plants['Tolerant'] = plants['Tolerant'].str.replace('Drought Salt','Drought, Salt')

In [ ]:
plants['Neglect Tolerant'] = 'No'
plants.loc[plants['Tolerant'].str.find('Drought') > -1,'Neglect Tolerant'] = 'Yes'

## Toxicity

In [ ]:
# Translate toxicity to common terms default on higher rating (for safety)
plants['Toxicity'] = plants['Toxicity'].str.title()
plants.loc[plants['Toxicity'].isin(['Mildly-Toxic','Mildly Toxic','Low Severity','Low']),'Toxicity'] = 'Mild'
plants.loc[plants['Toxicity'].isin(['Moderately Toxic','Medium Severity']),'Toxicity'] = 'Moderate'
plants.loc[plants['Toxicity'].isin(['Toxic','Toxic To Humans, Horses, Dogs, And Cats','Toxic To Dogs, Toxic To Cats',]),'Toxicity'] = 'High'
plants.loc[plants['Toxicity'].isin(['Non-Toxic','No','Non-Toxic To Pets']),'Toxicity'] = 'None'
plants['Toxicity'] = plants['Toxicity'].str.replace('Severely Toxic','Severe')

# Some straglers I looked up
plants.loc[plants['Common Name'] == 'Flaming Katy, Christmas kalanchoe, Florist kalanchoe, Madagascar widow’s thrill','Toxicity'] = 'High'
plants.loc[plants['Common Name'] == 'Rat Tail Cactus','Toxicity'] = 'Mild'
plants.loc[plants['Common Name'] == 'Rose Painted Calathea, Black Rose, Jungle rose','Toxicity'] = 'None'
plants.loc[plants['Common Name'] == 'Peanut Cactus','Toxicity'] = 'None'

## Hardiness zones

In [ ]:
# Split hardiness into two columns for min and max
# convert to same delimeter, split on spaces
plants['Hardiness (USDA)'] = plants['Hardiness (USDA)'].str.replace('\s?(-|to|–)\s?',' to ', regex=True)
plants[['Low Temp','High Temp']] = plants['Hardiness (USDA)'].str.split(' to ',expand=True)

# remove b from high and a from low
plants['Low Temp'] = plants['Low Temp'].str.replace('[Aa\s]','', regex=True)
plants['High Temp'] = plants['High Temp'].str.replace('[Bb\s]','', regex=True)

# low temp dictionary
temp_dict_low = {'3':-40,'4':-30,'5':-20,'6':-10,'7':0,'7b':5,'8':10,'9':20,'9b':25,'10':30,'10b':35,'11':40,'':''}

# high temp dictionary
temp_dict_high = {'8':20,'9':30,'10':40,'11a':45,'11':50,'12':60,'13':70,'':''}

# translate with the dictionaries to degrees F
plants['Low Temp'] = plants['Low Temp'].fillna('')
plants['Low Temp'] = [temp_dict_low[x] for x in list(plants['Low Temp'])]
plants['High Temp'] = plants['High Temp'].fillna('')
plants['High Temp'] = [temp_dict_high[x] for x in list(plants['High Temp'])]

## Further cleanup on blooming
This isn't used in the final analysis but it might be nice to have

In [ ]:
# clean up blooming seasons
plants['Bloom season'] = plants['Bloom season'].str.replace('Fall','Autumn')
plants['Bloom season'] = plants['Bloom season'].str.replace('\([Rr]are.*\)',', Rare', regex=True)
plants['Bloom season'] = plants['Bloom season'].str.replace('\s?\(.*\)','', regex=True)
plants['Bloom season'] = plants['Bloom season'].str.replace('(Late |Early )','', regex=True)
plants.loc[plants['Bloom season'].isin(['Year-round','Autumn, Spring, Summer, Winter',
                                        'Any time of the year']),'Bloom season'] = 'Spring, Summer, Autumn, Winter'
plants['Bloom season'] = plants['Bloom season'].str.replace('\s,',',', regex=True)

# probably a better way to put them in order, order doesn't even matter really
plants['Bloom season'] = plants['Bloom season'].str.replace('Winter, Spring', 'Spring, Winter')
plants['Bloom season'] = plants['Bloom season'].str.replace('Winter, Summer', 'Summer, Winter')
plants['Bloom season'] = plants['Bloom season'].str.replace('Summer, Spring', 'Spring, Summer')

# rarely
plants['Bloom season'] = plants['Bloom season'].str.replace('Rarely flowers indoors', 'Rare')
plants['Bloom season'] = plants['Bloom season'].str.replace(' and', ',')

# months to seasons
plants['Bloom season'] = plants['Bloom season'].str.replace('April to December', 'Spring, Summer, Autumn')
plants['Bloom season'] = plants['Bloom season'].str.replace('May to Jun', 'Summer')
plants['Bloom season'] = plants['Bloom season'].str.replace('September to October', 'Autumn')
plants['Bloom season'] = plants['Bloom season'].str.replace('July to August', 'Summer')
plants['Bloom season'] = plants['Bloom season'].str.replace('Easter to Christmas', 'Spring, Summer, Autumn')
plants['Bloom season'] = plants['Bloom season'].str.replace('Summere', 'Summer')

In [ ]:
# take care of the ones that just say "seasonal"
plants.loc[plants['Common Name'] == 'Rose Painted Calathea, Black Rose, Jungle rose','Bloom season'] = 'Rare'
plants.loc[plants['Common Name'].isin(['Bush Lily, Natal Lily, Kaffir Lily',
                                       'Jade Plant, Money Plant, Lucky Plant, Money Tree']),'Bloom season'] = 'Spring, Winter'
plants.loc[plants['Common Name'].isin(['Pineapple, bananas, pina','Watermelon Peperomia, Watermelon Begonia',
                                       'Crown of Thorns, Christ Plant, Christ Thorn',
                                       'Ghost Plant, Mother of Pearl Plant']),'Bloom season'] = 'Spring, Summer'
plants.loc[plants['Botanical Name'] == 'Beaucarnea recurvata','Bloom season'] = 'Rare'
plants.loc[plants['Botanical Name'] == 'Dracaena marginata','Bloom season'] = 'Spring, Summer'


## Uncomment and run this cell to save to csv

In [ ]:
#plants.to_csv('plants_clean.csv')